## Criar o Assistente

In [43]:

import openai
import os
from dotenv import find_dotenv, load_dotenv

# load the api key
_ = load_dotenv(find_dotenv())
client = openai.Client(api_key=os.getenv("API_KEY"))

### criando o assistente

In [44]:
assistant = client.beta.assistants.create(
    name="Tutor de matemática",
    instructions="Você é um tutor de matemática e deve ajudar os alunos a resolver problemas de matemática. Escreva e execute o código para resolver os problemas.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-3.5-turbo-0125"
)

### Criando uma Thread

In [45]:
thread = client.beta.threads.create()

In [46]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Qual a probabilidade de obter exatamente 150 vezes o número 6 ao lançar um dado honesto 1000 vezes aproximadamente?"
)

### Rodando a threead

In [47]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="O nome do usuário é Wellington e ele é um usuário Premium."
)

### Aguardart a thread rodar

In [48]:
import time
# roda várias vezes para identificar o status
while run.status in  ["queued", "in_progress", "cancelling" ]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
    print(run.status)

in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
in_progress
completed


### Quando terminar de roddar

In [49]:
if run.status == "completed":
    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
    )
    print(messages)
else:
    print(run.error)

SyncCursorPage[Message](data=[Message(id='msg_TXVJ7s0cqwa1dHfmBBiulQt7', assistant_id='asst_4YMqdzsS8y1uCNzzM9xh8Wk5', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='A probabilidade de obter exatamente 150 vezes o número 6 ao lançar um dado honesto 1000 vezes é aproximadamente 0.0126, ou seja, cerca de 1.26%.'), type='text')], created_at=1741382033, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_a8EWsgIwpZQgEn0s88WtqCoy', status=None, thread_id='thread_EosjVcaa2jeMvDrXqzp3xHbY'), Message(id='msg_nzefKRtv1GS6mgtq3H7VRKVA', assistant_id='asst_4YMqdzsS8y1uCNzzM9xh8Wk5', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Para calcular a probabilidade de obter exatamente 150 vezes o número 6 ao lançar um dado honesto 1000 vezes, podemos usar a distribuição binomial. \n\nA distribuição binomial é dada pela fórmula:\n\n\\[ P(X = k) = \\bin

In [50]:
print(messages.data[0].content[0].text.value)

A probabilidade de obter exatamente 150 vezes o número 6 ao lançar um dado honesto 1000 vezes é aproximadamente 0.0126, ou seja, cerca de 1.26%.


### Analisando os passos do modelo

In [51]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [72]:
run_steps.data[0].step_details

MessageCreationStepDetails(message_creation=MessageCreation(message_id='msg_TXVJ7s0cqwa1dHfmBBiulQt7'), type='message_creation')

In [82]:
for step in run_steps.data[::-1]:
    print("#step:", step.step_details.type)
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("> tool_call type:", tool_call.type)
            print("> input:")
            print(tool_call.code_interpreter.input)
            print("> outputs:", tool_call.code_interpreter.outputs)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id, message_id=step.step_details.message_creation.message_id
        )
        print("--- message_text:", message.content[0].text.value)


#step: message_creation
--- message_text: Para calcular a probabilidade de obter exatamente 150 vezes o número 6 ao lançar um dado honesto 1000 vezes, podemos usar a distribuição binomial. 

A distribuição binomial é dada pela fórmula:

\[ P(X = k) = \binom{n}{k} \times p^k \times (1-p)^{n-k} \]

onde:
- \( n \) é o número total de tentativas (neste caso, 1000 lançamentos),
- \( k \) é o número de sucessos desejados (neste caso, 150 vezes o número 6),
- \( p \) é a probabilidade de sucesso em uma única tentativa (neste caso, a probabilidade de obter o número 6 em um dado justo, que é 1/6).

Vamos calcular a probabilidade agora.
#step: tool_calls
> tool_call type: code_interpreter
> input:
from math import comb

# Número de tentativas
n = 1000

# Número de sucessos desejados (obter o número 6)
k = 150

# Probabilidade de sucesso em uma única tentativa
p = 1/6

# Calculando a probabilidade usando a fórmula da distribuição binomial
probability = comb(n, k) * (p ** k) * ((1-p) ** (n - k))
